In [ ]:

import numpy as np 
import pandas as pd
import seaborn as sns 
import warnings 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, accuracy_score, classification_report, confusion_matrix, f1_score, recall_score, precision_score
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from scipy.stats import randint
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import AdaBoostClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
warnings.filterwarnings('ignore')


In [20]:
df = pd.read_excel('filtered_blood_sugar_data.xlsx')

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1120 entries, 0 to 1119
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   AGE             1120 non-null   int64
 1   GENDER          1120 non-null   int64
 2   BLOOD PRESSURE  1120 non-null   int64
 3   BLOOD SUGAR     1120 non-null   int64
 4   PID FUN         1120 non-null   int64
 5   CLASS ID        1120 non-null   int64
dtypes: int64(6)
memory usage: 52.6 KB


In [22]:
df.describe()

,AGE,GENDER,BLOOD PRESSURE,BLOOD SUGAR,PID FUN,CLASS ID
count,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000
mean,55.435714,0.535714,102.819643,198.658929,0.003571,0.707143
std,11.753477,0.498946,31.730680,93.265118,0.059681,0.455277
min,24.000000,0.000000,52.000000,24.000000,0.000000,0.000000
25%,46.000000,0.000000,92.000000,135.000000,0.000000,0.000000
50%,55.000000,1.000000,100.000000,179.000000,0.000000,1.000000
75%,65.000000,1.000000,111.000000,243.000000,0.000000,1.000000
max,93.000000,1.000000,1034.000000,1101.000000,1.000000,1.000000


In [14]:
X=df.drop('CLASS ID',axis=1)
y=df['CLASS ID']

In [15]:
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.2,random_state=0)#splitting data in 80% train, 20%test

In [16]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [23]:
#Stacking Classifier for AdaBoost and Naive Bayes
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import RandomizedSearchCV

# Define base models (AdaBoost and Naive Bayes)
adaboost = AdaBoostClassifier(n_estimators=100, learning_rate=0.1)
nb_classifier = GaussianNB()

# Define meta-model (Logistic Regression)
meta_model = LogisticRegression()

# Create Stacking Classifier
stacked_model = StackingClassifier(estimators=[('adaboost', adaboost), ('naive_bayes', nb_classifier)], final_estimator=meta_model)

# Define hyperparameters to tune
param_dist = {
    'adaboost__n_estimators': [50, 100],
    'adaboost__learning_rate': [0.01, 0.1],
    'naive_bayes__var_smoothing': [1e-9, 1e-8],
    'final_estimator__C': [0.1, 1, 10]
}

# Hyperparameter tuning with RandomizedSearchCV
random_search = RandomizedSearchCV(stacked_model, param_dist, cv=5, scoring='accuracy', n_iter=10)

# Fit the model with training data
random_search.fit(X_train, y_train)

# Best estimator
best_stacked_model = random_search.best_estimator_

# Predict using the best model
y_pred = best_stacked_model.predict(X_test)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.8616

Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.74      0.72        53
           1       0.92      0.90      0.91       171

    accuracy                           0.86       224
   macro avg       0.81      0.82      0.81       224
weighted avg       0.86      0.86      0.86       224


Confusion Matrix:
[[ 39  14]
 [ 17 154]]


In [24]:
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import AdaBoostClassifier, StackingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from scipy.stats import uniform

# Suppress warnings
warnings.filterwarnings('ignore')

# Load dataset
df = pd.read_excel('filtered_blood_sugar_data.xlsx')

# Split features and target variable
X = df.drop('CLASS ID', axis=1)
y = df['CLASS ID']

# Split data into training and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define base models (AdaBoost and Naive Bayes)
adaboost = AdaBoostClassifier(n_estimators=100, learning_rate=0.1)
nb_classifier = GaussianNB()

# Define meta model (Logistic Regression)
meta_model = LogisticRegression()

# Create Stacking Classifier with base models and meta model
stacked_model = StackingClassifier(
    estimators=[('adaboost', adaboost), ('naive_bayes', nb_classifier)],
    final_estimator=meta_model
)

# Define hyperparameters to tune using RandomizedSearchCV
param_dist = {
    'adaboost__n_estimators': [50, 100],
    'adaboost__learning_rate': [0.01, 0.1],
    'naive_bayes__var_smoothing': [1e-9, 1e-8],
    'final_estimator__C': [0.1, 1, 10]
}

# Perform RandomizedSearchCV to find best hyperparameters
random_search = RandomizedSearchCV(stacked_model, param_dist, cv=5, scoring='accuracy', n_iter=10)
random_search.fit(X_train, y_train)

# Best estimator after RandomizedSearchCV
best_stacked_model = random_search.best_estimator_

# Predict using the best model
y_pred = best_stacked_model.predict(X_test)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Display classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Display confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Display best hyperparameters found during RandomizedSearchCV
print("\nBest Hyperparameters:")
print(random_search.best_params_)


Accuracy: 0.8616

Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.74      0.72        53
           1       0.92      0.90      0.91       171

    accuracy                           0.86       224
   macro avg       0.81      0.82      0.81       224
weighted avg       0.86      0.86      0.86       224


Confusion Matrix:
[[ 39  14]
 [ 17 154]]

Best Hyperparameters:
{'naive_bayes__var_smoothing': 1e-08, 'final_estimator__C': 0.1, 'adaboost__n_estimators': 50, 'adaboost__learning_rate': 0.01}


In [1]:
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import AdaBoostClassifier, StackingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.compose import ColumnTransformer
from scipy.stats import uniform

# Suppress warnings
warnings.filterwarnings('ignore')

# Load dataset
df = pd.read_excel('filtered_blood_sugar_data.xlsx')

# Split features and target variable
X = df.drop('CLASS ID', axis=1)
y = df['CLASS ID']

# Split data into training and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Scale continuous features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  # Apply scaling to training data
X_test = scaler.transform(X_test)  # Apply the same transformation to test data

# Define base models
adaboost = AdaBoostClassifier(n_estimators=100, learning_rate=0.1)
nb_classifier = GaussianNB()

# Define meta model (Logistic Regression)
meta_model = LogisticRegression()

# Create Stacking Classifier
stacked_model = StackingClassifier(
    estimators=[('adaboost', adaboost), ('naive_bayes', nb_classifier)],
    final_estimator=meta_model
)

# Define hyperparameters to tune
param_dist = {
    'adaboost__n_estimators': [50, 100],  # Number of estimators for AdaBoost
    'adaboost__learning_rate': [0.01, 0.1],  # Learning rate for AdaBoost
    'naive_bayes__var_smoothing': [1e-9, 1e-8],  # Variance smoothing for Naive Bayes
    'final_estimator__C': [0.1, 1, 10]  # Regularization strength for Logistic Regression
}

# Perform RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(
    estimator=stacked_model, 
    param_distributions=param_dist, 
    cv=5, 
    scoring='accuracy', 
    n_iter=10, 
    random_state=0
)
random_search.fit(X_train, y_train)

# Retrieve the best model
best_stacked_model = random_search.best_estimator_

# Make predictions
y_pred = best_stacked_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Display best hyperparameters
print("\nBest Hyperparameters:")
print(random_search.best_params_)


Accuracy: 0.8616

Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.74      0.72        53
           1       0.92      0.90      0.91       171

    accuracy                           0.86       224
   macro avg       0.81      0.82      0.81       224
weighted avg       0.86      0.86      0.86       224


Confusion Matrix:
[[ 39  14]
 [ 17 154]]

Best Hyperparameters:
{'naive_bayes__var_smoothing': 1e-08, 'final_estimator__C': 0.1, 'adaboost__n_estimators': 50, 'adaboost__learning_rate': 0.01}


In [ ]:
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import AdaBoostClassifier, StackingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Suppress warnings
warnings.filterwarnings('ignore')

# Load dataset
df = pd.read_excel('filtered_blood_sugar_data.xlsx')

# Split features and target variable
X = df.drop('CLASS ID', axis=1)
y = df['CLASS ID']

# Split data into training and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Scale continuous features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  # Apply scaling to training data
X_test = scaler.transform(X_test)  # Apply the same transformation to test data

# Define base models
adaboost = AdaBoostClassifier()
nb_classifier = GaussianNB()

# Define meta model (Logistic Regression)
meta_model = LogisticRegression()

# Create Stacking Classifier
stacked_model = StackingClassifier(
    estimators=[('adaboost', adaboost), ('naive_bayes', nb_classifier)],
    final_estimator=meta_model
)

# Define hyperparameter grid for GridSearchCV
param_grid = {
    'adaboost__n_estimators': [50, 100],  # Number of estimators for AdaBoost
    'adaboost__learning_rate': [0.01, 0.1],  # Learning rate for AdaBoost
    'naive_bayes__var_smoothing': [1e-9, 1e-8],  # Variance smoothing for Naive Bayes
    'final_estimator__C': [0.1, 1, 10]  # Regularization strength for Logistic Regression
}

# Perform GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(
    estimator=stacked_model,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    verbose=2,  # Optional: Increase verbosity to see progress
    n_jobs=-1  # Use all available CPU cores
)
grid_search.fit(X_train, y_train)

# Retrieve the best model
best_stacked_model = grid_search.best_estimator_

# Make predictions
y_pred = best_stacked_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Display best hyperparameters
print("\nBest Hyperparameters:")
print(grid_search.best_params_)


Fitting 5 folds for each of 24 candidates, totalling 120 fits
